In [2]:
import numpy as np
import xarray as xr
import cmocean.cm as cmo
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
from glob import glob
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import matplotlib.path as mpath
import matplotlib.ticker as mticker
from matplotlib.patches import ConnectionPatch
plt.rcParams['figure.facecolor'] = 'white'
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

In [ ]:
def prepro(ds):
    return ds.isel(y=slice(500, None))

Function to add latitude labels

In [3]:
def latitude_labels(ax, ds):    
    labels = [int(item._x) for item in ax.get_xticklabels()]
    labels_coord = [np.round(ds.nav_lat.isel(diag=label).values) for label in labels ]
    for il in range(0,len(labels)):
        if labels[il] % 200:
            labels_coord[il]=''
    ax.set_xticklabels(labels_coord)

Load grid and data files

In [ ]:
grid_files = ["/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/CREG12.L75-REF08_mask.nc", 
              "/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/CREG12.L75-REF08_mesh_hgr.nc",
              "/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/CREG12.L75-REF08_mesh_zgr.nc"]

In [ ]:
grid = xr.open_mfdataset(grid_files, parallel=True, preprocess=prepro)

In [4]:
ice = xr.open_mfdataset('/data0/project/drakkar/USERS/jrieck/transects/sithic_transect_0.nc')
iceREF = ice.sithic_ref_t0.mean("time_counter")
iceFUT = ice.sithic_fut_t0.mean("time_counter")
rho = xr.open_mfdataset("/data0/project/drakkar/USERS/jrieck/transects/density_transect_0.nc")
densityREF = rho.rhop_sig0_ref_t0.mean("time_counter")
densityFUT = rho.rhop_sig0_fut_t0.mean("time_counter")

In [5]:
data2plot_rho_t0 = xr.open_dataset('/data0/project/drakkar/USERS/jrieck/transects/density_transect_0.nc')
data2plot_rho_t1 = xr.open_dataset('/data0/project/drakkar/USERS/jrieck/transects/density_transect_1.nc')

PV_REF = xr.open_dataset('/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-PVORT/transect_1.nc')
PV_FUT = xr.open_dataset('/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-PVORT/transect_1.nc')

PV_REF_t2 = xr.open_dataset('/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-PVORT/transect_2.nc')
PV_FUT_t2 = xr.open_dataset('/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-PVORT/transect_2.nc')

rho0 = 1025.
data2plot_REF_PV = PV_REF.vototvor.differentiate('diag').drop('diag').interpolate_na("diag") / rho0
data2plot_FUT_PV = PV_FUT.vototvor.differentiate('diag').drop('diag').interpolate_na("diag") / rho0

In [6]:
data2plot_REF_PV_t0  = data2plot_REF_PV
data2plot_FUT_PV_t0  = data2plot_FUT_PV

diag_transect_wp_bp_t0 = xr.open_dataset('/data0/project/drakkar/USERS/jrieck/transects/Wp_Bp_transect_0.nc')
diag_transect_wp_bp_t1 = xr.open_dataset('/data0/project/drakkar/USERS/jrieck/transects/Wp_Bp_transect_1.nc')

data2plot_REF_wp_bp = diag_transect_wp_bp_t0.__xarray_dataarray_variable___ref_t0
data2plot_FUT_wp_bp = diag_transect_wp_bp_t0.__xarray_dataarray_variable___fut_t0

mean_WpBp_REF = xr.open_dataset("/data0/project/drakkar/USERS/jrieck/wp_bp_REF_clim_y500.nc")
mean_WpBp_FUT = xr.open_dataset("/data0/project/drakkar/USERS/jrieck/wp_bp_FUT_clim_y500.nc")

Integrate $w'b'$ over the top 300 m

In [237]:
deptht = mean_WpBp_REF.sel(deptht=slice(0, 300)).deptht

T_bc_REF = mean_WpBp_REF.sel(deptht=slice(0, 300)).sum("deptht") * deptht.max()
T_bc_FUT = mean_WpBp_FUT.sel(deptht=slice(0, 300)).sum("deptht") * deptht.max()

T_bc_REF = T_bc_REF.where(T_bc_REF!=0)
T_bc_FUT = T_bc_FUT.where(T_bc_FUT!=0)

Define some modifications to be made to all the maps plotted

In [8]:
def map_config(ax):
    ax.set_extent([-180, 180, 67, 90], ccrs.PlateCarree())
    ax.add_feature(cfeature.LAND, facecolor="silver")
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False, y_inline=True,
                     linewidth=1, color='gray', alpha=0.7, linestyle='--')
    plt.draw()
    return

Define `cartopy` projection to be used on maps

In [9]:
proj = ccrs.RotatedPole(pole_longitude=180, pole_latitude=40, central_rotated_longitude=0)

Plot Fig. 4

In [ ]:
fig = plt.figure(figsize=(10, 6))
gs = fig.add_gridspec(3, 24, height_ratios=[1, 0.05, 0.45])

# define axes
ax1 = fig.add_subplot(gs[0, 0:8], projection=proj)
ax2 = fig.add_subplot(gs[0, 8:16], projection=proj)
ax3 = fig.add_subplot(gs[0, 16:24], projection=proj)
ax4 = fig.add_subplot(gs[2, 0:8])
ax5 = fig.add_subplot(gs[2, 8:16])
ax6 = fig.add_subplot(gs[2, 16:24])

# plot maps of integrated w'b'
t1 = T_bc_REF.__xarray_dataarray_variable__.plot(x="nav_lon", y="nav_lat", ax=ax1, 
                                            norm=colors.SymLogNorm(linthresh=1e-5, linscale=1, 
                                                                    vmin=-1e-4, vmax=1e-4), cmap=cmo.delta, 
                                            add_colorbar=False, transform=ccrs.PlateCarree(),
                                            rasterized=True
                                           )

t2 = T_bc_FUT.__xarray_dataarray_variable__.plot(x="nav_lon", y="nav_lat", ax=ax2, 
                                            norm=colors.SymLogNorm(linthresh=1e-5, linscale=1, 
                                                                    vmin=-1e-4, vmax=1e-4), cmap=cmo.delta, 
                                            add_colorbar=False, transform=ccrs.PlateCarree(),
                                            rasterized=True
                                           )

t3 = (T_bc_FUT.__xarray_dataarray_variable__ - T_bc_REF.__xarray_dataarray_variable__).plot(x="nav_lon", y="nav_lat", ax=ax3, 
                                            norm=colors.SymLogNorm(linthresh=1e-5, linscale=1, 
                                                                    vmin=-1e-4, vmax=1e-4), cmap=cmo.delta, 
                                            add_colorbar=False, transform=ccrs.PlateCarree(),
                                            rasterized=True
                                           )

# plot transects of w'b' with density contours
w1 = data2plot_REF_wp_bp.plot(x='diag', ax=ax4, yincrease=False,rasterized=True, add_colorbar=False, 
                                                      norm=colors.SymLogNorm(linthresh=1e-10, linscale=1, 
                                                                    vmin=-1e-8, vmax=1e-8), cmap=cmo.delta)
cs4 = densityREF.where(densityREF> 1000).plot.contour(x='diag', yincrease=False, ax=ax4, 
                                                      vmin=1020, vmax=1030,
                                                      cmap=cmo.dense, levels=[1024,1026,1027])
ax4.clabel(cs4, inline=1, fontsize=10)
ice_ax4 = ax4.inset_axes([0, 1, 1, 0.2])
ice_ax4.fill_between(iceREF.diag, iceREF * 0, iceREF, alpha=0.5)

w2 = data2plot_FUT_wp_bp.plot(x='diag', ax=ax5, yincrease=False,rasterized=True, add_colorbar=False, 
                                                      norm=colors.SymLogNorm(linthresh=1e-10, linscale=1, 
                                                                    vmin=-1e-8, vmax=1e-8), cmap=cmo.delta)
cs5 = densityFUT.where(densityFUT>1000).plot.contour(x='diag', yincrease=False, ax=ax5, 
                                                      vmin=1020, vmax=1030,
                                                      cmap=cmo.dense, levels=[1024,1026,1027])
ax5.clabel(cs5, inline=1, fontsize=10)
ice_ax5 = ax5.inset_axes([0, 1, 1, 0.2])
ice_ax5.fill_between(iceFUT.diag, iceFUT * 0, iceFUT, alpha=0.5)

w3 = (data2plot_FUT_wp_bp - data2plot_REF_wp_bp).plot(x='diag', ax=ax6, 
                                                      yincrease=False,rasterized=True, add_colorbar=False, 
                                                      norm=colors.SymLogNorm(linthresh=1e-10, linscale=1, 
                                                                    vmin=-1e-8, vmax=1e-8), cmap=cmo.delta)

# configure maps
gl = [map_config(ax) for ax in [ax1, ax2, ax3]];

# add lines of transect to map
[ax.plot(densityREF.nav_lon, densityREF.nav_lat, transform=ccrs.PlateCarree(), color="dimgray", linewidth=2, zorder=6) 
 for ax in [ax1, ax2, ax3]]
[ax.plot(densityREF.nav_lon, densityREF.nav_lat, transform=ccrs.PlateCarree(), color="gold", zorder=7) 
 for ax in [ax1, ax2, ax3]]

# set limits, labels, ticks etc.
[ax.set_xlim(0, len(densityREF.diag)) for ax in [ax4, ax5, ax6]]
[ax.set_ylim(370, 0) for ax in [ax4, ax5, ax6]]
[ax.set_xlim((0, max(iceREF.diag))) for ax in [ice_ax4, ice_ax5]]
[ax.set_ylim((0, 3)) for ax in [ice_ax4, ice_ax5]]
[ax.set_yticks([2.5]) for ax in [ice_ax4, ice_ax5]]
[ax.set_xticks(np.arange(0, len(densityREF.diag), 100)) for ax in [ax4, ax5, ax6]]
[ax.xaxis.set_ticklabels([]) for ax in [ice_ax4, ice_ax5]]
[ax.yaxis.set_ticklabels([]) for ax in [ax5, ax6, ice_ax5]]
[ax.set_ylabel("") for ax in [ax5, ax6]]
[ax.set_ylabel("depth m") for ax in [ax4]]
[ax.set_ylabel("m") for ax in [ice_ax4]]
[ax.set_xlabel(r"latitude ($^{\circ}$North)") for ax in [ax4, ax5, ax6]]

[latitude_labels(ax, densityREF) for ax in [ax4, ax5, ax6]]

ax1.set_title("REF (1996-2015)", fontsize=16, fontweight="bold", pad=10)
ax2.set_title("FUT (2051-2070)", fontsize=16, fontweight="bold", pad=10)
ax3.set_title("FUT - REF", fontsize=16, fontweight="bold", pad=10)

[ax.text(-15, -100, t, fontsize=12) 
 for ax, t in zip([ax4, ax5, ax6], ["(d)", "(e)", "(f)"])]
[ax.text(-143, 65, t, fontsize=12, transform=ccrs.PlateCarree(), backgroundcolor="whitesmoke") 
 for ax, t in zip([ax1, ax2, ax3], 
                  ["(a)", "(b)", "(c)"])]

cax2 = ax3.inset_axes([1.05, 0, 0.05, 1])
cbar2 = plt.colorbar(t3, cax=cax2, orientation='vertical', extend="both")
cbar2.ax.tick_params(axis='y', which='major', pad=12)
cbar2.set_label(r"w'b' (m$^{3}\,$s$^{-3}$)")
cax2.set_yticks([-1e-4, -1e-5, 0, 1e-5, 1e-4])
cax2.set_yticklabels(["-1e-4", "-1e-5", "0",
                       "1e-5", "1e-4"], rotation=0, ha='center')

cax3 = ax6.inset_axes([1.05, 0, 0.05, 1])
cbar3 = plt.colorbar(w3, cax=cax3, orientation='vertical', extend="both")
cbar3.ax.tick_params(axis='y', which='major', pad=17)
cbar3.set_label(r"w'b' (m$^{2}\,$s$^{-3}$)")
cax3.set_yticks([-1e-8, -1e-9, -1e-10, 0, 1e-10, 1e-9, 1e-8])
cax3.set_yticklabels(["-10$^{-8}$", "-10$^{-9}$", "-10$^{-10}$", "0",
                       "10$^{-10}$", "10$^{-9}$", "10$^{-8}$"], rotation=0, ha='center')

for ax in [ax1, ax2, ax3]:
    ax.text(-30, 85, r"85$^{\circ}$N", transform=ccrs.PlateCarree(), ha="center", fontsize=7)
    ax.text(-31, 80, r"80$^{\circ}$N", transform=ccrs.PlateCarree(), ha="center", fontsize=7)
    ax.text(-34, 75, r"75$^{\circ}$N", transform=ccrs.PlateCarree(), ha="center", fontsize=7)
    ax.text(-37, 70, r"70$^{\circ}$N", transform=ccrs.PlateCarree(), ha="center", fontsize=7)
    ax.text(-41, 65, r"65$^{\circ}$N", transform=ccrs.PlateCarree(), ha="center", fontsize=7)

plt.subplots_adjust(wspace=1, hspace=0.1, bottom=0.15, top=0.9, left=0.1, right=0.9)

plt.savefig("figures/Figure_4_wb.png", dpi=600)

Plot Fig. S3

In [ ]:
fig = plt.figure(figsize=(10, 2.6))
gs = fig.add_gridspec(1, 24, height_ratios=[1])

ax01 = fig.add_subplot(gs[0, 0:8])
ax02 = fig.add_subplot(gs[0, 8:16])
ax03 = fig.add_subplot(gs[0, 16:24])

d1 = data2plot_REF_PV.plot(x='diag', yincrease=False, ax=ax01, rasterized=True,
                           add_colorbar=False, vmin=-2e-4, vmax=2e-4, cmap=cmo.balance)
cs1 = densityREF.where(densityREF> 1000).plot.contour(x='diag', yincrease=False, ax=ax01, 
                                                      vmin=1020, vmax=1030,
                                                      cmap=cmo.dense, levels=[1024,1026,1027])
ax01.clabel(cs1, inline=1, fontsize=10)
ice_ax1 = ax01.inset_axes([0, 1, 1, 0.2])
ice_ax1.fill_between(iceREF.diag, iceREF * 0, iceREF, alpha=0.5)

d2 = data2plot_FUT_PV.plot(x='diag', yincrease=False, ax=ax02, rasterized=True,
                           add_colorbar=False, vmin=-2e-4, vmax=2e-4, cmap=cmo.balance)
cs2 = densityFUT.where(densityFUT>1000).plot.contour(x='diag', yincrease=False, ax=ax02, 
                                                      vmin=1020, vmax=1030,
                                                      cmap=cmo.dense, levels=[1024,1026,1027])
ax02.clabel(cs2, inline=1, fontsize=10)
ice_ax2 = ax02.inset_axes([0, 1, 1, 0.2])
ice_ax2.fill_between(iceFUT.diag, iceFUT * 0, iceFUT, alpha=0.5)

d3 = (data2plot_FUT_PV - data2plot_REF_PV).plot(x='diag', yincrease=False, ax=ax03, rasterized=True,
                                         add_colorbar=False, vmin=-2e-4, vmax=2e-4, cmap=cmo.balance)


[ax.set_xlim(0, len(densityREF.diag)) for ax in [ax01, ax02, ax03]]
[ax.set_ylim(370, 0) for ax in [ax01, ax02, ax03]]
[ax.set_xlim((0, max(iceREF.diag))) for ax in [ice_ax1, ice_ax2]]
[ax.set_ylim((0, 3)) for ax in [ice_ax1, ice_ax2]]
[ax.set_yticks([2.5]) for ax in [ice_ax1, ice_ax2]]
[ax.set_xticks(np.arange(0, len(densityREF.diag), 100)) for ax in [ax01, ax02, ax03]]
[ax.xaxis.set_ticklabels([]) for ax in [ice_ax1, ice_ax2]]
[ax.yaxis.set_ticklabels([]) for ax in [ax02, ax03, ice_ax2]]
[ax.set_ylabel("") for ax in [ax02, ax03]]
[ax.set_ylabel("depth m") for ax in [ax01]]
[ax.set_ylabel("m") for ax in [ice_ax1]]
[ax.set_xlabel(r"latitude ($^{\circ}$North)") for ax in [ax01, ax02, ax03]]

[latitude_labels(ax, densityREF) for ax in [ax01, ax02, ax03]]

ax01.set_title("REF (1996-2015)", fontsize=16, fontweight="bold", pad=10)
ax02.set_title("FUT (2051-2070)", fontsize=16, fontweight="bold", pad=10)
ax03.set_title("FUT - REF", fontsize=16, fontweight="bold", pad=10)

[ax.text(-15, -100, t, fontsize=12) 
 for ax, t in zip([ax01, ax02, ax03], ["(a)", "(b)", "(c)"])]

cax1 = ax03.inset_axes([1.05, 0, 0.05, 1])
cbar1 = plt.colorbar(d1, cax=cax1, orientation='vertical', extend="both")
cax1.set_yticks([-2e-4, -1e-4, 0, 1e-4, 2e-4])
cax1.set_yticklabels(["-2e-4", "-1e-4", "0", "1e-4", "2e-4"])
cbar1.ax.tick_params(axis='y', which='major', pad=5)
cbar1.set_label(r"$\partial PV / \partial y$ (m$^{-2}$s$^{-1}$)")


plt.subplots_adjust(wspace=1, hspace=0.1, bottom=0.2, top=0.7, left=0.1, right=0.9)

plt.savefig("figures/Figure_S3_PV.png", dpi=600)